<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
data = """\
category\tdate1\tdate2      
 blue\t1/1/2018   
 blue\t\t1/2/2018
 blue\t\t1/5/2018
 blue\t2/1/2018
 green\t1/3/2018 
 green\t\t1/8/2018
 red\t12/1/2018
 red\t\t11/1/2018
 red\t\t12/5/2018
"""

# Find minimum date in date1 for the date1 for given category if it falls
# within 7 days.
# e.g. for blue 1/1 of date1, required date2 is 1/2.
#e.g. for blue 2/1/2018, it is nan

required = """\
  category      date1  isDateWithin7Days?     min_date
0     blue 01-01-2018                True   01-02-2018
1     blue 02-01-2018                False         NaN
2    green 01-03-2018                True   01-08-2018
3      red 12-01-2018                True   12-05-2018

"""

In [2]:
with open('dates.csv','w') as fo:
    fo.write(data)

In [3]:
%%bash
cat dates.csv

category	date1	date2      
 blue	1/1/2018   
 blue		1/2/2018
 blue		1/5/2018
 blue	2/1/2018
 green	1/3/2018 
 green		1/8/2018
 red	12/1/2018
 red		11/1/2018
 red		12/5/2018


In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('dates.csv',sep=r'\t',engine='python')

df['date1'] = pd.to_datetime(df['date1'], format = '%m/%d/%Y',errors='ignore')
df['date2'] = pd.to_datetime(df['date2'], format = '%m/%d/%Y',errors='ignore')
df

,category,date1,date2
0,blue,2018-01-01,NaT
1,blue,NaT,2018-01-02
2,blue,NaT,2018-01-05
3,blue,2018-02-01,NaT
4,green,2018-01-03,NaT
5,green,NaT,2018-01-08
6,red,2018-12-01,NaT
7,red,NaT,2018-11-01
8,red,NaT,2018-12-05


In [13]:
# get date1 only
df1 = df.dropna(subset = ['date1']).drop(columns = ['date2'])
df1

,category,date1
0,blue,2018-01-01
3,blue,2018-02-01
4,green,2018-01-03
6,red,2018-12-01


In [14]:
# get date2 only
df2 = df.dropna(subset = ['date2']).drop(columns = ['date1'])
df2

,category,date2
1,blue,2018-01-02
2,blue,2018-01-05
5,green,2018-01-08
7,red,2018-11-01
8,red,2018-12-05


In [15]:
# merge them
df3 = df1.merge(df2, on = 'category')
df3

,category,date1,date2
0,blue,2018-01-01,2018-01-02
1,blue,2018-01-01,2018-01-05
2,blue,2018-02-01,2018-01-02
3,blue,2018-02-01,2018-01-05
4,green,2018-01-03,2018-01-08
5,red,2018-12-01,2018-11-01
6,red,2018-12-01,2018-12-05


In [8]:
df3['isDateWithin7Days?'] = df3['date2'].between(df3['date1'] - pd.Timedelta(days=7), 
                                                 df3['date1'] + pd.Timedelta(days=7))
df3

,category,date1,date2,isDateWithin7Days?
0,blue,2018-01-01,2018-01-02,True
1,blue,2018-01-01,2018-01-05,True
2,blue,2018-02-01,2018-01-02,False
3,blue,2018-02-01,2018-01-05,False
4,green,2018-01-03,2018-01-08,True
5,red,2018-12-01,2018-11-01,False
6,red,2018-12-01,2018-12-05,True


In [9]:
df4 = df3.copy()
min_dates = df4[df4['isDateWithin7Days?']].groupby(
    ['category', 'date1'])['date2'].min().reset_index().rename(
    columns = {'date2': 'min_date'})

min_dates

,category,date1,min_date
0,blue,2018-01-01,2018-01-02
1,green,2018-01-03,2018-01-08
2,red,2018-12-01,2018-12-05


In [10]:
df3 = df3.groupby(
    ['category', 'date1'])['isDateWithin7Days?'].sum().reset_index()
df3

,category,date1,isDateWithin7Days?
0,blue,2018-01-01,2.0
1,blue,2018-02-01,0.0
2,green,2018-01-03,1.0
3,red,2018-12-01,1.0


In [11]:
df3['isDateWithin7Days?'] = np.where(df3['isDateWithin7Days?'] > 0, True, False)
df3

,category,date1,isDateWithin7Days?
0,blue,2018-01-01,True
1,blue,2018-02-01,False
2,green,2018-01-03,True
3,red,2018-12-01,True


In [12]:
df3.merge(min_dates, how = 'left', on = ['category', 'date1'])

,category,date1,isDateWithin7Days?,min_date
0,blue,2018-01-01,True,2018-01-02
1,blue,2018-02-01,False,NaT
2,green,2018-01-03,True,2018-01-08
3,red,2018-12-01,True,2018-12-05
